In [ ]:
import pandas as pd
data = pd.read_csv('Hyderbad_House_price.csv')
data


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3660 entries, 0 to 3659
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               3660 non-null   int64  
 1   title            3660 non-null   object 
 2   location         3660 non-null   object 
 3   price(L)         3660 non-null   float64
 4   rate_persqft     3660 non-null   int64  
 5   area_insqft      3660 non-null   int64  
 6   building_status  3660 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 200.3+ KB


In [ ]:
data=data.drop('id',axis=1)


In [5]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
category = ['title','location','building_status']

for col in category:
    data[col] = le.fit_transform(data[col])

    mapping_df = pd.DataFrame({
        'category': le.classes_,
        'code': le.transform(le.classes_)
    })
    mapping_df.to_csv(f"{col}_mapping.csv", index=False)


In [6]:
data

,title,location,price(L),rate_persqft,area_insqft,building_status
0,8,244,108.00,6000,1805,3
1,8,29,85.80,5500,1560,3
2,4,81,55.64,5200,1070,3
3,4,256,60.48,4999,1210,3
4,8,166,113.00,5999,1900,3
...,...,...,...,...,...,...
3655,4,16,60.00,6000,1000,1
3656,6,146,65.00,3250,2000,1
3657,5,280,65.00,6500,1000,1
3658,5,41,55.00,5500,1000,1


In [7]:
x = data.drop(['price(L)','rate_persqft'],axis = 1)
x

,title,location,area_insqft,building_status
0,8,244,1805,3
1,8,29,1560,3
2,4,81,1070,3
3,4,256,1210,3
4,8,166,1900,3
...,...,...,...,...
3655,4,16,1000,1
3656,6,146,2000,1
3657,5,280,1000,1
3658,5,41,1000,1


In [8]:
y = data[['price(L)','rate_persqft']]
y

,price(L),rate_persqft
0,108.00,6000
1,85.80,5500
2,55.64,5200
3,60.48,4999
4,113.00,5999
...,...,...
3655,60.00,6000
3656,65.00,3250
3657,65.00,6500
3658,55.00,5500


In [9]:
from sklearn.model_selection import train_test_split as tts
x_train,x_test,y_train,y_test = tts(x,y,test_size=0.3)


In [10]:
x_train

,title,location,area_insqft,building_status
3132,24,315,1323,0
1511,8,242,1920,3
1069,8,201,1780,3
2695,24,214,1620,0
310,8,255,1351,1
...,...,...,...,...
790,24,349,2700,0
2774,24,298,1026,0
419,24,60,2970,2
3434,11,166,1700,3


In [11]:
x_test

,title,location,area_insqft,building_status
3389,8,56,1655,3
2647,24,298,1494,0
2119,4,29,1210,1
1089,0,269,610,1
2091,8,170,2300,3
...,...,...,...,...
1873,24,47,900,0
688,10,317,2780,3
512,24,152,1485,0
1301,24,4,183,0


In [12]:
y_train

,price(L),rate_persqft
3132,22.05,1666
1511,126.00,6600
1069,137.00,7750
2695,14.39,888
310,64.00,4737
...,...,...
790,32.99,1222
2774,19.38,1888
419,264.00,8888
3434,228.00,13411


In [13]:
y_test

,price(L),rate_persqft
3389,67.87,4101
2647,8.32,556
2119,66.55,5500
1089,48.00,7869
2091,195.00,8500
...,...,...
1873,15.00,1666
688,208.00,7500
512,18.97,1278
1301,26.53,14500


In [14]:
from sklearn.ensemble import RandomForestRegressor
rfc = RandomForestRegressor()
rfc_model = rfc.fit(x_train,y_train)
rfc_model

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",100
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsample

In [15]:
rfc_predict = rfc_model.predict(x_test)
rfc_predict

array([[7.51607000e+01, 4.53929000e+03],
       [1.85659871e+01, 1.24264702e+03],
       [5.94581000e+01, 4.94305333e+03],
       ...,
       [1.95383578e+01, 1.31649733e+03],
       [4.46435000e+01, 2.61210333e+04],
       [2.20000000e+01, 1.10000000e+04]], shape=(1098, 2))

In [16]:
from sklearn.metrics import r2_score
rfc_r2 = r2_score(y_test, rfc_predict)
rfc_r2

0.39547434925019986

In [17]:
from lightgbm import LGBMRegressor
lgbm = LGBMRegressor()
lgbm_model = lgbm.fit(x_train, y_train['price(L)'])
lgbm_model

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000143 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 490
[LightGBM] [Info] Number of data points in the train set: 2562, number of used features: 4
[LightGBM] [Info] Start training from score 108.601807


,boosting_type,'gbdt'
,num_leaves,31
,max_depth,-1
,learning_rate,0.1
,n_estimators,100
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [18]:
lgbm_predict = lgbm_model.predict(x_test)
lgbm_predict

array([92.38863088, 22.7611416 , 60.09866611, ..., 19.82373297,
       30.26035387, 29.86470362], shape=(1098,))

In [19]:
lgbm_r2 = r2_score(y_test['price(L)'], lgbm_predict)
lgbm_r2

0.6028488647273074

In [20]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor

xgb = XGBRegressor()
multi_xgb = MultiOutputRegressor(xgb)
multi_xgb_model = multi_xgb.fit(x_train, y_train)
multi_xgb_model

,estimator estimator: estimator objectAn estimator object implementing :term:`fit` and :term:`predict`.,"XGBRegressor(...ree=None, ...)"
,"n_jobs n_jobs: int or None, optional (default=None)The number of jobs to run in parallel.:meth:`fit`, :meth:`predict` and :meth:`partial_fit` (if supportedby the passed estimator) will be parallelized for each target.When individual estimators are fast to train or predict,using ``n_jobs > 1`` can result in slower performance dueto the parallelism overhead.``None`` means `1` unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all available processes / threads.See :term:`Glossary ` for more details... versionchanged:: 0.20 `n_jobs` default changed from `1` to `None`.",None
,"objective objective: str | xgboost.sklearn._SklObjWProto | typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]] | NoneSpecify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'reg:squarederror'
,"base_score base_score: float | typing.List[float] | NoneThe initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.List[xgboost.callback.TrainingCallback] | NoneList of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: float | NoneSubsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: float | NoneSubsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: float | NoneSubsample ratio of columns when constructing each tree.,None
,"device device: str | None.. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: int | None.. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None


In [21]:
multi_xgb_predict = multi_xgb_model.predict(x_test)
multi_xgb_predict

array([[7.9652237e+01, 4.7172729e+03],
       [2.1216684e+01, 1.3821492e+03],
       [5.4052063e+01, 4.8058643e+03],
       ...,
       [1.5670762e+01, 9.4896429e+02],
       [3.9980671e+01, 2.1479850e+04],
       [2.3533800e+01, 1.1081130e+04]], shape=(1098, 2), dtype=float32)

In [22]:
multi_xgb_r2 = r2_score(y_test, multi_xgb_predict)
multi_xgb_r2

0.4489293694496155

In [23]:
print("rfc" if rfc_r2 > lgbm_r2 else "lgbm")

lgbm


In [24]:
import pickle
with open('multi_xgb_model', 'wb') as file:
    pickle.dump(multi_xgb_model, file)
    print(type(multi_xgb_model),"Model saved successfully.")

<class 'sklearn.multioutput.MultiOutputRegressor'> Model saved successfully.


In [54]:
import pandas as pd

title_map_list = pd.read_csv(r"D:\python\hydhouse\title_mapping.csv")

code = 1

result = title_map_list.loc[title_map_list['code'] == code, 'category']

print(result)


1    1 BHK Independent House
Name: category, dtype: object
